In [1]:
%pip install fake_useragent

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: fake_useragent in c:\programdata\anaconda3\envs\kidul\lib\site-packages (1.1.3)



In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
import re
import os
import json
import csv
import random
import datetime
from fake_useragent import UserAgent

ua = UserAgent()
headers = {"User-Agent" : ua.random}

In [3]:
dfphotographer=pd.read_csv('output/dfphotographer.csv')
dfphotographer.drop(columns = ['Unnamed: 0'], inplace = True)
display(dfphotographer)

,url,prize,photographer,year
0,https://www.worldpressphoto.org/collection/pho...,1955 Photo Contest,Mogens Haven,1955
1,https://www.worldpressphoto.org/collection/pho...,"Singles, 1st prize",Bela Zola,1956
2,https://www.worldpressphoto.org/collection/pho...,"Singles, 2nd prize",Fred Fischer,1956
3,https://www.worldpressphoto.org/collection/pho...,"Singles, 1st prize",Bela Zola,1956
4,https://www.worldpressphoto.org/collection/pho...,"Stories, 1st prize",Helmuth Pirath,1956
...,...,...,...,...
2866,https://www.worldpressphoto.org/collection/pho...,2023 Photo Contests,Mads Nissen,2023
2867,https://www.worldpressphoto.org/collection/pho...,2023 Photo Contests,Evgeniy Maloletka,2023
2868,https://www.worldpressphoto.org/collection/pho...,2023 Photo Contests,Chad Ajamian,2023
2869,https://www.worldpressphoto.org/collection/pho...,2023 Photo Contests,Evgeniy Maloletka,2023


In [4]:
def description_images(url,imagetitle,awardtitle,category,issue_date,information,prize_rank):
    page=requests.get(url,headers=headers)
    print(page.status_code)
    soup = BeautifulSoup(page.content, 'html.parser')
    try:
        titles = soup.find('h5', class_ = 'title title--16 title--gray').text
        titles = titles.replace(", ",",")
        titles_list = titles.split(",")
        if len(titles_list) == 3:
            awardtitle.append(titles_list[0])
            prize_rank.append(titles_list[2])
            category.append(titles_list[1])
        elif len(titles_list) == 2:
            awardtitle.append(titles_list[0])
            prize_rank.append(titles_list[1])
            category.append("Not Specified")
        else:
            awardtitle.append("Not Specified")
            prize_rank.append(titles_list[0])
            category.append("Not Specified")
    except:
        awardtitle.append("Not Specified")
        prize_rank.append("Not Specified")
        category.append("Not Specified")    
    try:
        info = soup.find('div', class_="photo-details__section photo-details__section--general-info")
        contents = [p.get_text() for p in info.find_all('p')]
        issued_date = contents[0]
        contents_info = contents[1].strip()
    except:
        issued_date = "Not Specified"
        contents_info = "Not Specified"

    try:
        phototitle = soup.find("h1", class_="title title--40 title--white").text
    except:
        phototitle = "Not Specified"
    
    imagetitle.append(phototitle)
    issue_date.append(issued_date)
    information.append(str(contents_info))

    print(phototitle+" "+issued_date)

In [15]:
def url_images(url,image_url,imagetitle,awardtitle,category,issue_date,information,prize_rank,year_path,photographer_path,image_storation,year_image,photographer_image,index_number):
    #Download images
    urltest_down = url
    i = 1
    while True:
        headers = {"User-Agent" : ua.random}
        urltest_down = urltest_down[0:len(urltest_down)-1]
        urltest_down = urltest_down+str(i)
        pageimage=requests.get(urltest_down,headers=headers)
        print(pageimage.status_code)
        soup = BeautifulSoup(pageimage.content, 'html.parser')
        try:
            wrongcode = soup.find('div', class_ ='container-middle').text
            wrongcode = wrongcode.replace("\r\n","")
            wrongcode = wrongcode.replace('\n','')
            wrongcode = wrongcode.replace('  ','')
            print(wrongcode)
            break       
        except:
            try:
                imgurl = soup.find('div', class_ = 'swiper-wrapper').find('div')['style']
                imgurl = re.findall(r"'(.+?)'", imgurl)
                result_imgurl = imgurl[0]

                imagetest = "https://www.worldpressphoto.org"+result_imgurl
                print(imagetest)
                pathurl = imagetest

                responseimage = requests.get(imagetest)

                pathimage = 'F:\\Temp\\'+str(year_path)+'\\'+str(photographer_path)

                if not os.path.exists(pathimage):
                    os.makedirs(pathimage)
                pathimagejpg = pathimage+'\\'+str(i)+'.jpg'
                print(pathimagejpg)

                with open(pathimagejpg, 'wb') as f:
                    f.write(responseimage.content)
                pathimage = 'F:\\Temp\\'+str(year_path)+'\\'+str(photographer_path)+'\\'+str(i)+'.jpg'

                description_images(urltest_down,imagetitle,awardtitle,category,issue_date,information,prize_rank)
                
                image_url.append(pathurl)
                image_storation.append(pathimage)
                year_image.append(year_path)
                photographer_image.append(photographer_path)
                index_number.append(i)
                i = i + 1

            except:
                pathurl = 'Not Found'
                pathimage = 'Not Found'
                image_url.append(pathurl)
                image_storation.append(pathimage)
                break


In [6]:
def description_images_2022_2023(url,phototitle_2023,issued_date_2023,background_2023,story_2023,awardtitle_2023,awardregion_2023,awardcategory_2023,jury_2023,year_image,photographer_image):
    page=requests.get(url,headers=headers)
    print(page.status_code)
    soup = BeautifulSoup(page.content, 'html.parser')
    try:
        phototitle = soup.find("h1", class_="title title--72 title--white title--font-normal").text
        phototitle = phototitle.strip()
        phototitle = phototitle.replace("  ","")
        phototitle = phototitle.replace("/r","")
    except:
        phototitle = "Not Specified"

    try:
        article = soup.find('article', class_ = 'photo-details-new__content').text
        article = article.replace("\n","/")
        article = article.replace("\r","/")
        article = article.replace("//","")
        article = article.replace("  ","")
        articlelist = article.split("/")
        issued_date = articlelist[0]
        background = articlelist[1]
        story = articlelist[2]
        
    except:
        issued_date = "Not Specified"
        background = "Not Specified"
        story = "Not Specified"

    try:
        contest = soup.find('div', class_ = 'photo-details-new__top-info-title').text
        contest = contest.strip()
        contest = contest.replace("  ","")
        contest = contest.replace(", ",",")
        contest = contest.replace("\n\r\n",",")
        contestlist = contest.split(",")
        awardtitle = contestlist[0]
        awardregion = contestlist[1]
        awardcategory = contestlist[2]
    except:
        awardtitle = "Not Specified"
        awardregion = "Not Specified"
        awardcategory = "Not Specified"

    try:
        jury = soup.find('article', class_='photo-details-new__jury').text
        jury = jury.replace("\n","")
        jury = jury.replace("\r","")
        jury = jury.replace("  ","")
        jury = jury.replace("Jury comment","")
    except:
        jury = "Not Specified"

    phototitle_2023.append(phototitle)
    issued_date_2023.append(issued_date)
    background_2023.append(background)
    story_2023.append(story)
    awardtitle_2023.append(awardtitle)
    awardregion_2023.append(awardregion)
    awardcategory_2023.append(awardcategory)
    jury_2023.append(jury)

    print(phototitle+" "+issued_date)

In [7]:
def download_images2022_2023(url,image_url_2023,phototitle_2023,issued_date_2023,background_2023,story_2023,awardtitle_2023,awardregion_2023,awardcategory_2023,jury_2023,year_path,photographer_path,image_storation_2023,year_image,photographer_image):
    #Download images
    urltest_down = url
    i = 1
    while True:
        headers = {"User-Agent" : ua.random}
        urltest_down = urltest_down[0:len(urltest_down)-1]
        urltest_down = urltest_down+str(i)
        pageimage=requests.get(urltest_down,headers=headers)
        print(pageimage.status_code)
        soupimage = BeautifulSoup(pageimage.content, 'html.parser')
        try:
            wrongcode = soupimage.find('div', class_ ='container-middle').text
            wrongcode = wrongcode.replace("\r\n","")
            wrongcode = wrongcode.replace('\n','')
            wrongcode = wrongcode.replace('  ','')
            print(wrongcode)
            break       
        except:
            try:
                imgurl = soupimage.find('section', class_ = 'photo-details-new__img-wrapper').find('img',class_='js-can-full-screen')['src']
                result_imgurl = imgurl

                imagetest = "https://www.worldpressphoto.org"+result_imgurl
                print(imagetest)
                pathurl = imagetest
                pathimage = 'F:\\Temp\\'+str(year_path)+'\\'+str(photographer_path)+'\\'+str(i)+'.jpg'
                
                responseimage = requests.get(imagetest)

                pathimage = 'F:\\Temp\\'+str(year_path)+'\\'+str(photographer_path)

                if not os.path.exists(pathimage):
                    os.makedirs(pathimage)
                pathimagejpg = pathimage+'\\'+str(i)+'.jpg'
                print(pathimagejpg)

                with open(pathimagejpg, 'wb') as f:
                    f.write(responseimage.content)
                
                i = i + 1

                description_images_2022_2023(urltest_down,phototitle_2023,issued_date_2023,background_2023,story_2023,awardtitle_2023,awardregion_2023,awardcategory_2023,jury_2023)
                image_url_2023.append(pathurl)
                image_storation_2023.append(pathimage)
                year_image.append(year_path)
                photographer_image.append(photographer_path)
            except:
                pathurl = 'Not Found'
                pathimage = 'Not Found'
                description_images_2022_2023(url,phototitle_2023,issued_date_2023,background_2023,story_2023,awardtitle_2023,awardregion_2023,awardcategory_2023,jury_2023)
                image_url_2023.append(pathurl)
                image_storation_2023.append(pathimage)
                break


In [8]:
def image_scrape(year):
    imagetitle = []
    awardtitle = []
    category = []
    issue_date = []
    information=[]
    prize_rank = []
    year_image = []
    photographer_image = []
    image_url = []
    image_storation = []
    index_number = []

    dfyear = dfphotographer[dfphotographer['year'] == year]
    dfyear

    for link in dfyear['url']:
        print(link)
        year_path = year
        print(year_path)
        photographer_path = dfyear['photographer'][dfyear['url']==link].values[0]
        photographer_path = photographer_path.strip()
        print(photographer_path)
        url_images(link,image_url,imagetitle,awardtitle,category,issue_date,information,prize_rank,year_path,photographer_path,image_storation,year_image,photographer_image,index_number)

    dfimageurl = pd.DataFrame({'Index':index_number,'Title':imagetitle,'Award':awardtitle,'Category':category,'Published Date':issue_date,'URL':image_url,'Stored Path':image_storation,'Description':information,'Prize Level':prize_rank,'Photographer':photographer_image,'Award Year':year_image})
    display(dfimageurl)
    dfimageurl.to_csv('output/dfimageurl'+str(year)+'.csv')

In [9]:
def image_scrape_2022_2023(year):
    phototitle_2023=[]
    issued_date_2023=[]
    background_2023=[]
    story_2023=[]
    awardtitle_2023=[]
    awardregion_2023=[]
    awardcategory_2023=[]
    jury_2023=[]
    year_image = []
    photographer_image = []
    image_url_2023 = []
    image_storation_2023 = []

    dfyear = dfphotographer[dfphotographer['year'] == year]
    dfyear

    for link in dfyear['url']:
        print(link)
        year_path = year
        print(year_path)
        photographer_path = dfyear['photographer'][dfyear['url']==link].values[0]
        photographer_path = photographer_path.strip()
        print(photographer_path)
        download_images2022_2023(link,image_url_2023,phototitle_2023,issued_date_2023,background_2023,story_2023,awardtitle_2023,awardregion_2023,awardcategory_2023,jury_2023,year_path,photographer_path,image_storation_2023,year_image,photographer_image)

    dfimageurl2022_2023 = pd.DataFrame({'Title':phototitle_2023,'Published Date':issued_date_2023,'Background':background_2023,'Story':story_2023,'Award':awardtitle_2023,'Award Region':awardregion_2023,'Award Category':awardcategory_2023,'Jury':jury_2023,'URL':image_url_2023,'Stored Path':image_storation_2023,'Photographer':photographer_image,'Award Year':year_image})
    dfimageurl2022_2023.to_csv('output/dfimageurl'+str(year)+'.csv')

In [ ]:
for year in range(2003,2022):    
    image_scrape(year)

#### **Combined all csv**

In [17]:
import os
import pandas as pd

folder_path = 'output/dfimageurl/'
merged_df = pd.DataFrame()

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        merged_df = pd.concat([merged_df, df], ignore_index=True)
        merged_df.sort_values(by=['Award Year'], inplace=True)
        merged_df.drop(['Unnamed: 0'], axis=1, inplace=True)
        merged_df.reset_index(drop=True, inplace=True)

merged_df.to_csv('dfimageurlall.csv', index=False)


In [20]:
dfimageurlall = pd.read_csv('dfimageurlall1.csv')
dfimageurlall

,Index,Title,Award,Category,Published Date,URL,Stored Path,Description,Prize Level,Photographer,Award Year,Background,Story,Award Region,Award Category,Jury
0,1,Not Specified,1955 Photo Contest,Not Specified,"28 August, 1955",https://www.worldpressphoto.org/getmedia/7ce98...,F:\Temp\1955\Mogens Haven\1.jpg,A competitor tumbles off his motorcycle during...,World Press Photo of the Year,Mogens Haven,1955,NaN,NaN,NaN,NaN,NaN
1,1,Not Specified,1956 Photo Contest,Photo Stories,"01 January, 1956",https://www.worldpressphoto.org/getmedia/b8294...,F:\Temp\1956\Helmuth Pirath\1.jpg,‘Sp?theimkehrer’: A German prisoner of war is ...,1st prize,Helmuth Pirath,1956,NaN,NaN,NaN,NaN,NaN
2,1,Not Specified,1956 Photo Contest,Spot News,"01 January, 1956",https://www.worldpressphoto.org/getmedia/92600...,F:\Temp\1956\Louis Garnade\1.jpg,A child is being rescued by the fire brigade f...,2nd prize,Louis Garnade,1956,NaN,NaN,NaN,NaN,NaN
3,1,Not Specified,1956 Photo Contest,Spot News,"08 August, 1956",https://www.worldpressphoto.org/getmedia/c45dd...,F:\Temp\1956\Antoine Rulmont\1.jpg,Family members wait for news about their loved...,1st prize,Antoine Rulmont,1956,NaN,NaN,NaN,NaN,NaN
4,1,Not Specified,1956 Photo Contest,Sports,"31 January, 1956",https://www.worldpressphoto.org/getmedia/bb85f...,F:\Temp\1956\Franz Votava\1.jpg,The famous Austrian alpine skier Tony Sailer d...,3rd prize,Franz Votava,1956,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13138,5,Home for the Golden Gays,2023 Photo Contest,NaN,"18 July, 2022",https://www.worldpressphoto.org/getmedia/19364...,F:\Temp\2023\HannahReyes Morales\5.jpg,NaN,NaN,HannahReyes Morales,2023,Al Enriquez (86) looks through a curtain in th...,The Golden Gays are a community of older LGBTQ...,Southeast Asia and Oceania,Stories,This project enlists an intersectional approac...
13139,6,Home for the Golden Gays,2023 Photo Contest,NaN,"18 July, 2022",https://www.worldpressphoto.org/getmedia/a3249...,F:\Temp\2023\HannahReyes Morales\6.jpg,NaN,NaN,HannahReyes Morales,2023,Al Enriquez (86) looks through a curtain in th...,The Golden Gays are a community of older LGBTQ...,Southeast Asia and Oceania,Stories,This project enlists an intersectional approac...
13140,7,Home for the Golden Gays,2023 Photo Contest,NaN,"18 July, 2022",https://www.worldpressphoto.org/getmedia/fcbf6...,F:\Temp\2023\HannahReyes Morales\7.jpg,NaN,NaN,HannahReyes Morales,2023,Al Enriquez (86) looks through a curtain in th...,The Golden Gays are a community of older LGBTQ...,Southeast Asia and Oceania,Stories,This project enlists an intersectional approac...
13141,1,Shifting,2023 Photo Contest,NaN,Not Specified,https://www.worldpressphoto.org/getmedia/5520f...,F:\Temp\2023\Johanna Alarcón\1.jpg,NaN,NaN,Johanna Alarcón,2023,Valentina is a 13-year-old who aspires to beco...,"In Latin America, the incarceration of women h...",South America,Open Format,A beautifully executed project that approaches...
